<a href="https://colab.research.google.com/github/dzakialthalsy/1-Dzaki-Althalsyah/blob/main/%5BKlasifikasi%5D_Submission_Akhir_BMLP_Dzaki_Althalsyah.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Import library**

In [65]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from sklearn.metrics import classification_report
import joblib

# **2. Gunakan dataset hasil clustering yang memiliki fitur Target**

In [66]:
import matplotlib.pyplot as pltim
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [67]:
df = pd.read_csv('/content/drive/MyDrive/data_clustering_inverse.csv')

# **3. Tampilkan 5 baris pertama dengan function head**

In [68]:
df.head()

,TransactionAmount,TransactionType,Location,Channel,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,CustomerAge,Target
0,-1.107930,Credit,San Diego,ATM,Doctor,-0.537894,0.0,Low,Senior,0
1,0.579431,Credit,Houston,ATM,Doctor,0.304662,0.0,High,Senior,1
2,-0.585158,Credit,Mesa,Online,Student,-0.888959,0.0,Low,Young,2
3,-0.313941,Credit,Raleigh,Online,Student,-1.324280,0.0,Medium,Adult,0
4,-0.744226,Credit,Oklahoma City,ATM,Student,0.739983,0.0,NaN,Young,2


### MULAI CODE OPSIONAL ###

In [69]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1501 entries, 0 to 1500
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   TransactionAmount    1501 non-null   float64
 1   TransactionType      1501 non-null   object 
 2   Location             1501 non-null   object 
 3   Channel              1501 non-null   object 
 4   CustomerOccupation   1501 non-null   object 
 5   TransactionDuration  1501 non-null   float64
 6   LoginAttempts        1501 non-null   float64
 7   AccountBalance       1263 non-null   object 
 8   CustomerAge          1501 non-null   object 
 9   Target               1501 non-null   int64  
dtypes: float64(3), int64(1), object(6)
memory usage: 117.4+ KB


In [70]:
df.describe(include='all')

,TransactionAmount,TransactionType,Location,Channel,CustomerOccupation,TransactionDuration,LoginAttempts,AccountBalance,CustomerAge,Target
count,1.501000e+03,1501,1501,1501,1501,1.501000e+03,1501.0,1263,1501,1501.000000
unique,NaN,1,43,3,4,NaN,NaN,3,4,NaN
top,NaN,Credit,Charlotte,ATM,Student,NaN,NaN,Low,Adult,NaN
freq,NaN,1501,49,593,400,NaN,NaN,866,596,NaN
mean,-4.733796e-17,NaN,NaN,NaN,NaN,8.284142e-18,0.0,NaN,NaN,1.521652
std,1.000333e+00,NaN,NaN,NaN,NaN,1.000333e+00,0.0,NaN,NaN,1.109518
min,-1.172368e+00,NaN,NaN,NaN,NaN,-1.534919e+00,0.0,NaN,NaN,0.000000
25%,-8.072665e-01,NaN,NaN,NaN,NaN,-7.906613e-01,0.0,NaN,NaN,1.000000
50%,-2.725665e-01,NaN,NaN,NaN,NaN,-1.025737e-01,0.0,NaN,NaN,2.000000
75%,5.277131e-01,NaN,NaN,NaN,NaN,6.135991e-01,0.0,NaN,NaN,3.000000


# **4. Menggunakan train_test_split() untuk melakukan pembagian dataset.**

In [71]:
X = df.drop('Target', axis=1)
y = df['Target']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"X_train shape: {X_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"y_test shape: {y_test.shape}")

X_train shape: (1200, 9)
X_test shape: (301, 9)
y_train shape: (1200,)
y_test shape: (301,)


# **5. Buatlah model klasifikasi menggunakan Decision Tree**

In [75]:
# Identifikasi kolom kategorikal
categorical_cols = X_train.select_dtypes(include='object').columns

# Terapkan One-Hot Encoding ke X_train dan X_test
X_train = pd.get_dummies(X_train, columns=categorical_cols, drop_first=True)
X_test = pd.get_dummies(X_test, columns=categorical_cols, drop_first=True)

# Pastikan kolom-kolom antara X_train dan X_test selaras setelah encoding
X_train, X_test = X_train.align(X_test, join='left', axis=1, fill_value=0)

print("Shape X_train setelah one-hot encoding:", X_train.shape)
print("Shape X_test setelah one-hot encoding:", X_test.shape)

display(X_train.head())

Shape X_train setelah one-hot encoding: (1200, 55)
Shape X_test setelah one-hot encoding: (301, 55)


,TransactionAmount,TransactionDuration,LoginAttempts,Location_Atlanta,Location_Austin,Location_Baltimore,Location_Boston,Location_Charlotte,Location_Chicago,Location_Colorado Springs,...,Channel_Branch,Channel_Online,CustomerOccupation_Engineer,CustomerOccupation_Retired,CustomerOccupation_Student,AccountBalance_Low,AccountBalance_Medium,CustomerAge_Middle-aged,CustomerAge_Senior,CustomerAge_Young
538,-0.887639,0.627642,0.0,False,False,False,False,False,False,False,...,True,False,False,True,False,True,False,False,True,False
1217,1.554807,-0.903002,0.0,False,False,False,False,False,False,False,...,True,False,False,False,True,True,False,False,False,False
1113,-1.063155,-1.225982,0.0,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,True,False,False
324,1.350776,0.079980,0.0,False,False,False,False,False,False,False,...,False,False,False,False,True,True,False,False,False,False
361,-0.530318,0.557429,0.0,False,False,False,False,True,False,False,...,True,False,True,False,False,True,False,True,False,False


In [76]:
dt_classifier = DecisionTreeClassifier(random_state=42)
dt_classifier.fit(X_train, y_train)

y_pred_dt = dt_classifier.predict(X_test)

accuracy_dt = accuracy_score(y_test, y_pred_dt)
precision_dt = precision_score(y_test, y_pred_dt, average='weighted')
recall_dt = recall_score(y_test, y_pred_dt, average='weighted')
f1_dt = f1_score(y_test, y_pred_dt, average='weighted')

print("Decision Tree Classifier Performance:")
print(f"Accuracy: {accuracy_dt:.4f}")
print(f"Precision: {precision_dt:.4f}")
print(f"Recall: {recall_dt:.4f}")
print(f"F1-Score: {f1_dt:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_dt))

Decision Tree Classifier Performance:
Accuracy: 0.9601
Precision: 0.9629
Recall: 0.9601
F1-Score: 0.9601

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        87
           2       0.97      0.87      0.92        75
           3       0.86      0.97      0.91        66

    accuracy                           0.96       301
   macro avg       0.96      0.96      0.96       301
weighted avg       0.96      0.96      0.96       301



# **6. Menyimpan Model**

In [77]:
joblib.dump(dt_classifier, 'decision_tree_model.h5')
print("Decision Tree model saved as decision_tree_model.h5")

Decision Tree model saved as decision_tree_model.h5


# **7. Melatih model menggunakan algoritma klasifikasi scikit-learn selain Decision Tree. (Contoh: RandomForestClassifier)**

In [78]:
rf_classifier = RandomForestClassifier(random_state=42)
rf_classifier.fit(X_train, y_train)

y_pred_rf = rf_classifier.predict(X_test)

accuracy_rf = accuracy_score(y_test, y_pred_rf)
precision_rf = precision_score(y_test, y_pred_rf, average='weighted')
recall_rf = recall_score(y_test, y_pred_rf, average='weighted')
f1_rf = f1_score(y_test, y_pred_rf, average='weighted')

print("Random Forest Classifier Performance:")
print(f"Accuracy: {accuracy_rf:.4f}")
print(f"Precision: {precision_rf:.4f}")
print(f"Recall: {recall_rf:.4f}")
print(f"F1-Score: {f1_rf:.4f}")

print("\nClassification Report:")
print(classification_report(y_test, y_pred_rf))

Random Forest Classifier Performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000

Classification Report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        87
           2       1.00      1.00      1.00        75
           3       1.00      1.00      1.00        66

    accuracy                           1.00       301
   macro avg       1.00      1.00      1.00       301
weighted avg       1.00      1.00      1.00       301



# **8. Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada seluruh algoritma yang sudah dibuat.**

In [79]:
print("\n--- Perbandingan Kinerja Model --- ")

print("\nDecision Tree Classifier:")
print(f"  Accuracy: {accuracy_dt:.4f}")
print(f"  Precision: {precision_dt:.4f}")
print(f"  Recall: {recall_dt:.4f}")
print(f"  F1-Score: {f1_dt:.4f}")
print("\nClassification Report for Decision Tree:")
print(classification_report(y_test, y_pred_dt))

print("\nRandom Forest Classifier (Untuned):")
print(f"  Accuracy: {accuracy_rf:.4f}")
print(f"  Precision: {precision_rf:.4f}")
print(f"  Recall: {recall_rf:.4f}")
print(f"  F1-Score: {f1_rf:.4f}")
print("\nClassification Report for Untuned Random Forest:")
print(classification_report(y_test, y_pred_rf))


--- Perbandingan Kinerja Model --- 

Decision Tree Classifier:
  Accuracy: 0.9601
  Precision: 0.9629
  Recall: 0.9601
  F1-Score: 0.9601

Classification Report for Decision Tree:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        87
           2       0.97      0.87      0.92        75
           3       0.86      0.97      0.91        66

    accuracy                           0.96       301
   macro avg       0.96      0.96      0.96       301
weighted avg       0.96      0.96      0.96       301


Random Forest Classifier (Untuned):
  Accuracy: 1.0000
  Precision: 1.0000
  Recall: 1.0000
  F1-Score: 1.0000

Classification Report for Untuned Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        87
           2       1.00      1.00      1.00        75
     

# **9. Menyimpan Model Selain Decision Tree**

In [80]:
joblib.dump(rf_classifier, 'explore_random forest_classification.h5')
print("Random Forest model saved as explore_random forest_classification.h5")

Random Forest model saved as explore_random forest_classification.h5


# **10. Lakukan Hyperparameter Tuning dan Latih ulang.**

### Tentukan Grid Hyperparameter

In [81]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_features': ['sqrt', 'log2'],
    'max_depth': [10, 20, 30, None],
    'criterion': ['gini', 'entropy']
}

print("Hyperparameter grid for RandomForestClassifier has been defined:")
print(param_grid)

Hyperparameter grid for RandomForestClassifier has been defined:
{'n_estimators': [100, 200, 300], 'max_features': ['sqrt', 'log2'], 'max_depth': [10, 20, 30, None], 'criterion': ['gini', 'entropy']}


### Lakukan Hyperparameter Tuning

In [82]:
rf_classifier_grid = RandomForestClassifier(random_state=42)

grid_search = GridSearchCV(estimator=rf_classifier_grid, param_grid=param_grid, cv=5, verbose=2, n_jobs=-1)
grid_search.fit(X_train, y_train)

print("Best parameters found:")
print(grid_search.best_params_)

print("Best cross-validation score:")
print(grid_search.best_score_)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best parameters found:
{'criterion': 'gini', 'max_depth': None, 'max_features': 'sqrt', 'n_estimators': 200}
Best cross-validation score:
0.9958333333333333


### Latih Ulang model Random Forest yang sudah di Tuning

In [83]:
best_params = grid_search.best_params_

rf_classifier_tuned = RandomForestClassifier(**best_params, random_state=42)
rf_classifier_tuned.fit(X_train, y_train)

y_pred_rf_tuned = rf_classifier_tuned.predict(X_test)

accuracy_rf_tuned = accuracy_score(y_test, y_pred_rf_tuned)
precision_rf_tuned = precision_score(y_test, y_pred_rf_tuned, average='weighted')
recall_rf_tuned = recall_score(y_test, y_pred_rf_tuned, average='weighted')
f1_rf_tuned = f1_score(y_test, y_pred_rf_tuned, average='weighted')

# **11. Menampilkan hasil evaluasi akurasi, presisi, recall, dan F1-Score pada algoritma yang sudah dituning.**

In [84]:
print("Tuned Random Forest Classifier Performance:")
print(f"Accuracy: {accuracy_rf_tuned:.4f}")
print(f"Precision: {precision_rf_tuned:.4f}")
print(f"Recall: {recall_erf_tuned:.4f}")
print(f"F1-Score: {f1_rf_tuned:.4f}")

print("\nClassification Report for Tuned Random Forest:")
print(classification_report(y_test, y_pred_rf_tuned))

Tuned Random Forest Classifier Performance:
Accuracy: 1.0000
Precision: 1.0000
Recall: 1.0000
F1-Score: 1.0000

Classification Report for Tuned Random Forest:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00        73
           1       1.00      1.00      1.00        87
           2       1.00      1.00      1.00        75
           3       1.00      1.00      1.00        66

    accuracy                           1.00       301
   macro avg       1.00      1.00      1.00       301
weighted avg       1.00      1.00      1.00       301



# **12. Menyimpan Model hasil tuning**

In [85]:
joblib.dump(rf_classifier_tuned, 'tuning_classifier.h5')
print("Tuned Random Forest model saved as tuning_classification.h5")

Tuned Random Forest model saved as tuning_classification.h5
